# compa

> Retrieve and process data from WRDS Compustat Annual

In [ ]:
#| default_exp wrds.compa

In [ ]:
#|exports
from __future__ import annotations
from typing import List

import pandas as pd
import numpy as np

import pandasmore as pdm
from finsets.wrds import wrds_api

In [ ]:
#| exports
PROVIDER = 'Wharton Research Data Services (WRDS)'
URL = 'https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-annual/'
LIBRARY = 'comp'
TABLE = 'funda'
COMPANY_TABLE = 'company' #contains some header information that is missing from comp.funda (e.g. sic and naics) 
FREQ = 'Y'
MIN_YEAR = 1950
MAX_YEAR = None
ENTITY_ID_IN_RAW_DSET = 'gvkey'
ENTITY_ID_IN_CLEAN_DSET = 'gvkey'
TIME_VAR_IN_RAW_DSET = 'datadate'
TIME_VAR_IN_CLEAN_DSET = f'{FREQ}date'

In [ ]:
#| export
def list_all_vars() -> pd.DataFrame:
    "Collects names of all available variables from WRDS f`{LIBRARY}.{TABLE}` and `{LIBRARY}.{COMPANY_TABLE}`."

    try:
        db = wrds_api.Connection()
        funda = db.describe_table(LIBRARY,TABLE).assign(wrds_library=LIBRARY, wrds_table=TABLE)
        fundn = db.describe_table(LIBRARY,COMPANY_TABLE).assign(wrds_library=LIBRARY, wrds_table=COMPANY_TABLE)
    finally:
        db.close()

    return pd.concat([funda,fundn])[['name','type','wrds_library','wrds_table']].copy()

In [ ]:
#| eval: false
all_vars = list_all_vars()

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.


In [ ]:
#| eval: false
all_vars.loc[all_vars.name.str.contains('conm'),:]

,name,type,wrds_library,wrds_table
9,conm,VARCHAR(70),comp,funda
0,conm,VARCHAR(70),comp,company
10,conml,VARCHAR(100),comp,company


In [ ]:
#| export
def default_raw_vars():
    """Defines default variables used in `get_raw_data` if none are specified."""

    return ['datadate', 'gvkey', 'cusip' ,'cik' ,'tic' ,'fyear' ,'fyr' , 'fic',
            'naicsh', 'sich' , 'sic', 'naics', 'exchg',  
            'lt' ,'at' ,'txditc' ,'pstkl' ,'pstkrv' ,'pstk' ,'csho' ,'ajex' , 'rdip',
            'act' ,'dvc' ,'xad','seq' ,'che' ,'lct' ,'dlc' ,'ib' ,'dvp' ,'txdi' ,'dp' ,
            'txp' ,'oancf' ,'ivncf' ,'fincf' ,'dltt' ,'mib','ceq' ,'invt' ,'cogs' , 'revt',
            'sale' ,'capx' ,'xrd' ,'txdb' ,'prcc_f' ,'sstk' ,'prstkc' ,'dltis' ,'dltr' ,'emp' ,
            'dd1' ,'ppegt' ,'ppent' ,'xint' ,'txt' ,'sppe' ,'gdwl' ,'xrent' ,'re' ,'dvpsx_f' ,
            'tstk' ,'wcap' ,'rect' ,'xsga' ,'aqc' ,'oibdp' ,'dpact','ni' ,'ivao' ,'ivst' ,
            'dv' , 'intan' ,'pi' ,'txfo' ,'pifo' ,'xpp' ,'drc' ,'drlt' ,'ap' ,'xacc' ,'itcb']             

In [ ]:
print(default_raw_vars())

['datadate', 'gvkey', 'cusip', 'cik', 'tic', 'fyear', 'fyr', 'fic', 'naicsh', 'sich', 'sic', 'naics', 'exchg', 'lt', 'at', 'txditc', 'pstkl', 'pstkrv', 'pstk', 'csho', 'ajex', 'rdip', 'act', 'dvc', 'xad', 'seq', 'che', 'lct', 'dlc', 'ib', 'dvp', 'txdi', 'dp', 'txp', 'oancf', 'ivncf', 'fincf', 'dltt', 'mib', 'ceq', 'invt', 'cogs', 'revt', 'sale', 'capx', 'xrd', 'txdb', 'prcc_f', 'sstk', 'prstkc', 'dltis', 'dltr', 'emp', 'dd1', 'ppegt', 'ppent', 'xint', 'txt', 'sppe', 'gdwl', 'xrent', 're', 'dvpsx_f', 'tstk', 'wcap', 'rect', 'xsga', 'aqc', 'oibdp', 'dpact', 'ni', 'ivao', 'ivst', 'dv', 'intan', 'pi', 'txfo', 'pifo', 'xpp', 'drc', 'drlt', 'ap', 'xacc', 'itcb']


In [ ]:
#| export
def parse_varlist(vars: List[str]=None,
                  required_vars = [],
                  ) -> str:
    """Figures out which `vars` come from the `{LIBRARY}.{TABLE}` table and which come from the `{LIBRARY}.{COMPANY_TABLE}` table and adds a. and b. prefixes to variable names to feed into an SQL query"""

    # Get all available variables and add suffixes needed for the SQL query
    suffix_mapping = {TABLE: 'a.', COMPANY_TABLE: 'b.'}
    all_avail_vars = list_all_vars().drop_duplicates(subset='name',keep='first').copy()
    all_avail_vars['w_prefix'] = all_avail_vars.apply(lambda row: suffix_mapping[row['wrds_table']] + row['name'] , axis=1)

    if vars == '*': return ','.join(list(all_avail_vars['w_prefix']))
    
    # Add required vars to requested vars
    if vars is None: vars = default_raw_vars()
    vars_to_get =  required_vars + [x for x in list(set(vars)) if x not in required_vars]

    # Validate variables to be downloaded (make sure that they are in the target database)
    invalid_vars = [v for v in vars_to_get if v not in list(all_avail_vars.name)]
    if invalid_vars: raise ValueError(f"These vars are not in the database: {invalid_vars}") 

    # Extract information on which variable comes from which wrds table, so we know what prefix to use
    vars_to_get = pd.DataFrame(vars_to_get, columns=['name'])
    get_these = vars_to_get.merge(all_avail_vars, how = 'left', on = 'name')
        
    return ','.join(list(get_these['w_prefix']))

In [ ]:
#| eval: false
parse_varlist(['at','lt','sic'])

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.


'a.lt,b.sic,a.at'

In [ ]:
#| export
def get_raw_data(
        vars: List[str]=None, # If None, downloads `default_raw_vars`; use '*' to get all available variables
        required_vars: List[str]=['gvkey','datadate'], #list of variables that will get downloaded, even if not in `vars`
        nrows: int=None, #Number of rows to download. If None, full dataset will be downloaded
        start_date: str=None, # Start date in MM/DD/YYYY format
        end_date: str=None #End date in MM/DD/YYYY format
) -> pd.DataFrame:
    """Downloads `vars` from `start_date` to `end_date` from WRDS `{LIBRARY}.{TABLE}` library """
 
    wrds_api.validate_dates([start_date, end_date])
    vars = parse_varlist(vars, required_vars=required_vars)

    sql_string=f"""SELECT  {vars}  
                    FROM {LIBRARY}.{TABLE} as a 
                    LEFT JOIN {LIBRARY}.{COMPANY_TABLE} as b ON a.gvkey = b.gvkey
                    WHERE  indfmt='INDL' AND datafmt='STD' AND popsrc='D' AND consol='C'
                """
    if start_date is not None: sql_string += r" AND datadate >= %(start_date)s"
    if end_date is not None: sql_string += r" AND datadate <= %(end_date)s"
    if nrows is not None: sql_string += r" LIMIT %(nrows)s"
    
    return wrds_api.download(sql_string,
                             params={'start_date':start_date, 'end_date':end_date, 'nrows':nrows})

The `get_raw_data` function will produce unique `gvkey-datadate` records, which is not the case for the data we would get from the WRDS [website](https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-annual/), because they use `indfmt IN ('INDL','FS')` (while we use `indfmt='INDL'`).

In [ ]:
#| eval: false
m = get_raw_data(['at','indfmt','sich','sic'],nrows=1)
m

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.
Loading library list...
Done


,gvkey,datadate,indfmt,sich,sic,at
0,001000,1961-12-31,INDL,<NA>,3089,<NA>


In [ ]:
#| eval: false
r = get_raw_data(vars='*', nrows=1)
r

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.
Loading library list...
Done


,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate,curr_sp500_flag
0,001000,1961-12-31,1961,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,3089,325,978,<NA>,<NA>,0,<NA>,1978-06-30,<NA>,0.0


In [ ]:
#| eval: false
raw = get_raw_data(vars = ['at', 'lt'], start_date='01/01/2021', end_date='01/01/2022', nrows=1)

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.
Loading library list...
Done


In [ ]:
#| eval: false
raw.head(0)

,gvkey,datadate,lt,at


In [ ]:
#| eval: false
raw = get_raw_data(start_date='01/01/2021', end_date='01/01/2023')

Loading library list...
Done
Approximately 921202 rows in comp.funda.
Approximately 55294 rows in comp.company.
Loading library list...
Done


In [ ]:
#| eval: false
raw.head(0)

,gvkey,datadate,dd1,naicsh,sstk,sic,txp,xsga,ivst,exchg,...,ib,xrd,dltis,drc,cik,ivncf,capx,txdb,pstk,ceq


In [ ]:
#| export
def process_raw_data(
        df: pd.DataFrame=None,  # Must contain `permno` and `datadate` columns   
        clean_kwargs: dict={},  # Params to pass to `pdm.setup_panel` other than `panel_ids`, `time_var`, and `freq`
) -> pd.DataFrame:
    """Applies `pandasmore.setup_panel` to `df`"""

    # Change some variables to categorical
    for col in ['gvkey','naics','sic','fic','cik','tic','cusip']:
        if col in df.columns:
            df[col] = df[col].astype('string').astype('category')

    if 'sich' in df.columns:
        df['sich'] = df['sich'].astype('Int64').astype('string').str.zfill(4).astype('category')

    if 'naicsh' in df.columns:
        df['naicsh'] = df['naicsh'].astype('Int64').astype('string').astype('category')

    # Set up panel structure
    df = pdm.setup_panel(df, panel_ids=ENTITY_ID_IN_RAW_DSET, time_var=TIME_VAR_IN_RAW_DSET, freq=FREQ, 
                         panel_ids_toint=False,
                         **clean_kwargs)
    return df 

In [ ]:
#| eval: false
df_clean = process_raw_data(raw)

In [ ]:
#| eval: false
df_clean.head(0)

,,datadate,dtdate,dd1,naicsh,sstk,sic,txp,xsga,ivst,exchg,...,ib,xrd,dltis,drc,cik,ivncf,capx,txdb,pstk,ceq
gvkey,Ydate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| export
def features(df: pd.DataFrame=None
             ) -> pd.DataFrame:

    out = pd.DataFrame(index=df.index)

    # industry 
    out['sic_full'] = df['sich'].astype('object').fillna(df['sic'].astype('object')).astype('category')
    out['naics_full'] = df['naicsh'].astype('object').fillna(df['naics'].astype('object')).astype('category')

    # size 
    out['stock_price'] = np.abs(df['prcc_f'])
    out['lag_at'] = pdm.lag(df[['at']])
    out['mktcap'] = out['stock_price'] * df['csho']

    # book equity vars
    out['pstk0'] = df['pstk'].fillna(0)
    out['pref_stock'] = np.where(df['pstkrv'].isnull(), df['pstkl'], df['pstkrv'])
    out['pref_stock'] = np.where(out['pref_stock'].isnull(),out['pstk0'], out['pref_stock'])
    out['shreq'] = np.where(df['seq'].isnull(), df['ceq'] + out['pstk0'], df['seq'])
    out['shreq'] = np.where(out['shreq'].isnull(), df['at'] - df['lt'] - df['mib'].fillna(0), out['shreq'])
    out['bookeq'] = out['shreq'] + df['txditc'].fillna(0) + df['itcb'].fillna(0) - out['pref_stock']
    #out['bookeq_w_itcb'] = out['bookeq'] + df['itcb'].fillna(0)

    out['tobinq'] = (df['at'] - out['bookeq'] + out['stock_price'] * df['csho']) / df['at']

    # issuance vars
    out['equityiss_tot'] = (pdm.rdiff(out['bookeq']) - pdm.rdiff(df['re'])) 
    out['equityiss_cfs'] = (df['sstk'].fillna(0) - df['prstkc'].fillna(0))
    out['debtiss_tot'] = (pdm.rdiff(df['at']) - pdm.rdiff(out['bookeq'])) 
    out['debtiss_cfs'] = (df['dltis'].fillna(0) - df['dltr'].fillna(0)) 
    out['debtiss_bs'] = (pdm.rdiff(df['dltt']) + pdm.rdiff(df['dlc'].fillna(0))) 
    for v in ['equityiss_tot','equityiss_cfs','debtiss_tot','debtiss_cfs','debtiss_bs']:
        out[f'{v}_2la'] = out[v] / out['lag_at']

    # investment vars
    out['ppent_pch'] = pdm.rpct_change(df['ppent'])
    out['capx_2la'] = df['capx'] / out['lag_at']

    # profitability vars
    out['roa'] = df['ib'] / df['at']

    # cash flow vars
    out['cflow_is'] = df['ib']+df['dp']
    out['cflow_cfs'] = df['oancf'] 
    out['cflow_full'] = np.where(df.dtdate.dt.year<=1987, out['cflow_is'], out['cflow_cfs'])
    for v in ['cflow_is','cflow_cfs','cflow_full']:
        out[f'{v}_2la'] = out[v] / out['lag_at']

    # liquidity vars
    out['cash_2a'] = df['che'] / df['at']

    # leverage vars
    out['booklev'] = (df['dltt'] + df['dlc']) / df['at']
    out.loc[out.booklev<0, 'booklev'] = 0
    out.loc[out.booklev>1, 'booklev'] = 1

    # payout vars
    out['dividends_2la'] = (df['dvc'].fillna(0) + df['dvp'].fillna(0)) / out['lag_at']
    out['repurchases_2la'] = (df['prstkc'].fillna(0) - pdm.rdiff(df['pstkrv']).fillna(0)) / out['lag_at']

    out = out.replace([np.inf, -np.inf], np.nan)
    return out 


In [ ]:
#| eval: false
ftrs = features(df_clean)

In [ ]:
#| eval: false
ftrs.head(0)

,,sic_full,naics_full,stock_price,lag_at,mktcap,pstk0,pref_stock,shreq,bookeq,tobinq,...,cflow_is,cflow_cfs,cflow_full,cflow_is_2la,cflow_cfs_2la,cflow_full_2la,cash_2a,booklev,dividends_2la,repurchases_2la
gvkey,Ydate,,,,,,,,,,,,,,,,,,,,,


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()